# Importando as bibliotecas

In [1]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np
from collections import Counter
import os
import sys
sys.path.insert(0, os.path.abspath('../Util'))
from dados import ProcessarDados

# Dataset

In [2]:
procData = ProcessarDados("../dataset/norm_bin_10_FEATURES_M17_CM6b_TH199.csv")

# Funções

In [3]:
# funcção que treina e testa o modelo armazenando as métricas
# retorna um dicionário cotendo os valores das métricas de cada rodada
def treinamento_teste(epocas = 10, k_folds = 5, exibir_matriz_confusao=False, exibir_metricas=False):
    
    #array para armazenar as das métricas de cada rodada
    resultados_accuracy = []
    resultados_precision = []
    resultados_recall = []
    resultados_f1 = []
    resultados_parametros = []
    
    #dicionário das métricas
    resultados_gerais = {}

    for i in range(epocas):
        # divisão os dados 
        seed = i
        X_train, X_test, y_train, y_test = procData.holdout(0.2, seed)
        #print(Counter(y_test))

        # realizando o grid search para encontrar a melhor combinação entre o C, gamma e Kernel, 
        # considerando a acurácia (taxa de acerto)
        # aqui o método GridSearchCV é configurado para subdividir os dados de treino em k_folds
        clf = svm.SVC(random_state = seed)
        grid_svm = GridSearchCV(clf, param_grid, cv=k_folds, scoring='accuracy', verbose=0)
        grid_svm.fit(X_train, y_train)

        # Treinando do modelo com os melhores parametros encontrados
        C_best = grid_svm.best_estimator_.C
        gamma_best = grid_svm.best_estimator_.gamma
        kernel_best = grid_svm.best_estimator_.kernel

        SVM = svm.SVC(random_state = seed, C = C_best, gamma = gamma_best, kernel = kernel_best)
        SVM.fit(X_train, y_train)

        #testando o modelo
        y_pred = SVM.predict(X_test)
        if exibir_matriz_confusao:
            print(confusion_matrix(y_test, y_pred))

        # calculado as metricas
        accuracy = metrics.accuracy_score(y_test, y_pred)
        precision = metrics.precision_score(y_test, y_pred)
        recall = metrics.recall_score(y_test, y_pred)
        f1_score = 2 * (precision * recall) / (precision + recall)

        # armazenando as métricas
        resultados_accuracy.append(accuracy)
        resultados_precision.append(precision)
        resultados_recall.append(recall)
        resultados_f1.append(f1_score)

        best_parametros = {"C": C_best, "Gamma": gamma_best, "Kernel": kernel_best};
        resultados_parametros.append(best_parametros)
        


        if exibir_metricas:
            print("Rodada: #",i)
            print(best_parametros)
            print("Accuracy:",accuracy)
            print("Precision:",precision)
            print("Recall:",recall)
            print("F1-Score:",f1_score)
            print("\n")

            
    resultados_gerais['accuracy'] = resultados_accuracy
    resultados_gerais['precision'] = resultados_precision
    resultados_gerais['recall'] = resultados_recall
    resultados_gerais['f1'] = resultados_f1
    resultados_gerais['params'] = resultados_parametros
    
    return resultados_gerais

In [4]:
def tabelaMetricas(nome_modelo, dict_metricas, rodadas=False, salvarResultados=True):
        
    print ("============================================== "+nome_modelo+" =================================================")
    print ("=================================== TABELA DE MÉTRICAS DO MODELO ===================================")
    
    if(rodadas==False):
        print ("\t Accuracy \t|\t Precision \t|\t Recall \t|\t F1-Score")
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['accuracy'], axis=0), np.std(dict_metricas['accuracy'], axis=0)),end='       ')
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['precision'], axis=0), np.std(dict_metricas['precision'], axis=0)),end='    ')
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['recall'], axis=0), np.std(dict_metricas['recall'], axis=0)),end='       ')
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['f1'], axis=0), np.std(dict_metricas['f1'], axis=0)))
        print ("====================================================================================================")
        
    if(salvarResultados):
        # save to npy file
        np.save('../resultados/resultados_'+nome_modelo+'.npy', dict_metricas)

# Definindo os parâmetros

In [5]:
c_range = [10, 20, 30, 40, 50]
gammas = [0.01, 0.02, 0.03, 0.04, 0.05]
kernels = ["rbf"]
param_grid = {'C': c_range, 'gamma': gammas, 'kernel': kernels}

epocas = 50
k_folds = 5
exibir_matriz_confusao = True
exibir_metricas = True
salvarResultados = True
rodadas=False

# Treinando e obtendo as métricas do modelo

In [6]:
# treinando o modelo
dict_metricas = treinamento_teste(epocas, k_folds, exibir_matriz_confusao, exibir_metricas)
tabelaMetricas('SVM',dict_metricas, rodadas, salvarResultados)

[[23  5]
 [ 3 89]]
Rodada: # 0
{'C': 40, 'Gamma': 0.03, 'Kernel': 'rbf'}
Accuracy: 0.9333333333333333
Precision: 0.9468085106382979
Recall: 0.967391304347826
F1-Score: 0.956989247311828


[[19 14]
 [ 3 84]]
Rodada: # 1
{'C': 10, 'Gamma': 0.01, 'Kernel': 'rbf'}
Accuracy: 0.8583333333333333
Precision: 0.8571428571428571
Recall: 0.9655172413793104
F1-Score: 0.908108108108108


[[14  8]
 [ 7 91]]
Rodada: # 2
{'C': 40, 'Gamma': 0.04, 'Kernel': 'rbf'}
Accuracy: 0.875
Precision: 0.9191919191919192
Recall: 0.9285714285714286
F1-Score: 0.9238578680203046


[[25  7]
 [ 2 86]]
Rodada: # 3
{'C': 20, 'Gamma': 0.03, 'Kernel': 'rbf'}
Accuracy: 0.925
Precision: 0.9247311827956989
Recall: 0.9772727272727273
F1-Score: 0.9502762430939227


[[21  7]
 [ 3 89]]
Rodada: # 4
{'C': 50, 'Gamma': 0.04, 'Kernel': 'rbf'}
Accuracy: 0.9166666666666666
Precision: 0.9270833333333334
Recall: 0.967391304347826
F1-Score: 0.9468085106382979


[[18 14]
 [ 3 85]]
Rodada: # 5
{'C': 30, 'Gamma': 0.05, 'Kernel': 'rbf'}
Accurac

[[20 13]
 [ 0 87]]
Rodada: # 45
{'C': 50, 'Gamma': 0.02, 'Kernel': 'rbf'}
Accuracy: 0.8916666666666667
Precision: 0.87
Recall: 1.0
F1-Score: 0.9304812834224598


[[18  7]
 [ 6 89]]
Rodada: # 46
{'C': 40, 'Gamma': 0.03, 'Kernel': 'rbf'}
Accuracy: 0.8916666666666667
Precision: 0.9270833333333334
Recall: 0.9368421052631579
F1-Score: 0.9319371727748692


[[15  9]
 [ 4 92]]
Rodada: # 47
{'C': 30, 'Gamma': 0.02, 'Kernel': 'rbf'}
Accuracy: 0.8916666666666667
Precision: 0.9108910891089109
Recall: 0.9583333333333334
F1-Score: 0.934010152284264


[[22  9]
 [ 0 89]]
Rodada: # 48
{'C': 10, 'Gamma': 0.01, 'Kernel': 'rbf'}
Accuracy: 0.925
Precision: 0.9081632653061225
Recall: 1.0
F1-Score: 0.9518716577540107


[[24 10]
 [ 2 84]]
Rodada: # 49
{'C': 50, 'Gamma': 0.04, 'Kernel': 'rbf'}
Accuracy: 0.9
Precision: 0.8936170212765957
Recall: 0.9767441860465116
F1-Score: 0.9333333333333332


============================================== SVM =================================================
=================